In [2]:
import json, requests, os
import logging
from shapely.geometry import shape, MultiLineString, LineString, Point
from rtree import index
from io import BytesIO
import geopandas as gpd

In [3]:
# URL of the GeoJSON file
url = 'https://github.com/newzealandpaul/Shipping-Lanes/blob/main/data/Shipping_Lanes_v1.geojson?raw=true'

# Fetch the GeoJSON file
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Read the GeoJSON data into a GeoDataFrame
geojson_data = BytesIO(response.content)
gdf = gpd.read_file(geojson_data)

# Display the first few rows of the GeoDataFrame
gdf.head()

,FID,OBJECTID,Type,geometry
0,0,1,Major,"MULTILINESTRING ((-34.8576 -8.13738, -34.67437..."
1,1,2,Middle,"MULTILINESTRING ((-52.14663 47.21993, -52.0640..."
2,2,3,Minor,"MULTILINESTRING ((100.43797 13.33198, 100.4257..."


In [4]:
# Load GeoJSON file
#with open('routes.geojson', 'r') as f:
#    data = json.load(f)

data = json.loads(response.content)
#print(data)

# Convert features to Shapely LineString objects
major = []
middle = []
minor = []

for feature in data['features']:
    route_type = feature['properties']['Type']
    routes = [] 
    geom = shape(feature['geometry'])
    if isinstance(geom, MultiLineString):
        for line in geom.geoms:
            routes.append(line)
    
    if route_type == 'Major':
        major = routes
    elif route_type == 'Middle':
        middle = routes
    elif route_type == 'Minor':
        minor = routes
    else:
        print(f"Unknown route type: {route_type}")

print (major)
print (middle)
print (minor)

[<LINESTRING (-34.858 -8.137, -34.674 -8.07, -34.674 -8.07, -25.63 16.957, -2...>, <LINESTRING (-165.501 53.937, -165.063 53.957, -164.9 53.964, -164.737 53.97...>, <LINESTRING (-166.55 54.004, -166.743 54.002, -166.796 54.002, -166.849 54.0...>, <LINESTRING (180 53.134, 179.985 53.132, 179.89 53.121, 179.8 53.109, 179.78...>, <LINESTRING (174.901 52.383, 174.573 52.332, 174.512 52.323, 174.451 52.313,...>, <LINESTRING (-53.299 46.63, -52.883 46.752, -52.695 46.807, -52.506 46.861, ...>, <LINESTRING (-180 50.559, -179.457 50.589, -179.335 50.595, -179.213 50.601,...>, <LINESTRING (179.904 50.553, 180 50.559)>, <LINESTRING (143.309 41.702, 144.097 42.067, 144.214 42.12, 144.33 42.173, 1...>, <LINESTRING (-122.774 37.712, -125.398 38.646, -125.385 39.202, -124.94 48.471)>, <LINESTRING (-9.485 36.599, -9.841 36.782, -9.974 36.85, -10.107 36.918, -10...>, <LINESTRING (-120.647 34.197, -120.647 34.197, -121.443 34.635, -121.539 34....>, <LINESTRING (-63.711 44.493, -53.735 46.531, -53.378 4

In [5]:
## Deviate a bit

# Create a KML file with all routes in major, middle, and minor
def create_kml(routes, name, color='ff0000ff'):  # Default color is red (ABGR format)
    kml = f"""<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
<Document>
    <name>{name}</name>
    <Style id="lineStyle">
        <LineStyle>
            <color>{color}</color>
            <width>4</width>
        </LineStyle>
    </Style>"""
    
    # Add each line as a separate Placemark
    for i, line in enumerate(routes):
        kml += f"""
    <Placemark>
        <name>{name} - Line {i+1}</name>
        <styleUrl>#lineStyle</styleUrl>
        <LineString>
            <coordinates>"""
            
        coords = list(line.coords)
        for coord in coords:
            kml += f"{coord[0]},{coord[1]},0 "
            
        kml += """</coordinates>
        </LineString>
    </Placemark>"""
    
    kml += """
</Document>
</kml>"""
    return kml

# Create KML files for each route type with different colors
major_kml = create_kml(major, "Major Routes", "ff0000ff")  # Red
middle_kml = create_kml(middle, "Middle Routes", "ff00ff00")  # Green
minor_kml = create_kml(minor, "Minor Routes", "ffff0000")  # Blue

# Save KML files
with open('major_routes.kml', 'w') as f:
    f.write(major_kml)
with open('middle_routes.kml', 'w') as f:
    f.write(middle_kml)
with open('minor_routes.kml', 'w') as f:
    f.write(minor_kml)

In [21]:
# Build a spatial index for the routes


def build_spatial_index(routes):
    idx = index.Index()
    for i, line in enumerate(routes):
        # Get the bounding box of the line
        #minx, miny, maxx, maxy = line.bounds
        # Insert the line into the spatial index
        #idx.insert(i, (minx, miny, maxx, maxy), obj=line)
        
        idx.insert(i, line.bounds, obj=line)
        
    return idx


major_idx = build_spatial_index(major)
middle_idx = build_spatial_index(middle)
minor_idx = build_spatial_index(minor)

In [22]:
def find_candidate_route(idx, lon, lat, distance_threshold=0.0001):
    query_point = Point(lon, lat)
    # Find the 5 nearest candidates (you can adjust the number)
    candidate_indices = list(idx.nearest((lon, lat, lon, lat), 5))
    for i in candidate_indices:
        route = routes[i]
        # Find the closest point on the route
        proj_distance = route.project(query_point)
        nearest_point = route.interpolate(proj_distance)
        # If within an acceptable distance, consider it a hit
        if query_point.distance(nearest_point) < distance_threshold:
            return i, route, proj_distance
    return None, None, None

In [35]:

i, route, proj_distance = find_candidate_route(major_idx, 52.146, 47.2199314, distance_threshold=100)
if i is not None:
    print(f"Found route {i} with distance {proj_distance}")
else:
    print("No route found within the distance threshold.")

Found route 34 with distance 46.364379551009314
